In [33]:
import json

In [34]:
data = json.loads(open('new_desc.json', encoding='utf-8').read())
data = [json.loads(x) for x in data]
endata = data[::2]
chdata = data[1::2]
assert len(endata) == len(chdata)

In [38]:
allres = {}
ename_to_cname = {}
for ed, cd in zip(endata, chdata):
    for k in ed.keys():
        if k[:5] == 'SKILL':
            ori_k = k
            k = k.replace('NORMAL ATTACK', 'NORMAL_ATTACK')
            k = k.replace('ELEMENTAL SKILL', 'ELEMENTAL_SKILL')
            k = k.replace('ELEMENTAL BURST', 'ELEMENTAL_BURST')
            k = k.replace('PASSIVE SKILL', 'PASSIVE_SKILL')
            mat = {
                'NORMAL_ATTACK': '普通攻击',
                'ELEMENTAL_SKILL': '元素战技',
                'ELEMENTAL_BURST': '元素爆发',
                'PASSIVE_SKILL': '被动技能',
            }
            c_skill_type = None
            for etype in mat:
                if etype in k:
                    c_skill_type = mat[etype]
                    break
            for ck in cd:
                if c_skill_type in ck:
                    break
            else:
                print('Cannot find', k, 'in', cd)
                continue
            ename_to_cname[k] = ck
            allres[k] = ed[ori_k]
            allres[k]['names'].update(cd[ck]['names'])
            version = list(allres[k]['descs'].keys())[0]
            allres[k]['descs'][version].update(cd[ck]['descs'][version])
        else:
            assert len(ed.keys()) == 1
            assert len(cd.keys()) == 1
            ck = list(cd.keys())[0]
            ename_to_cname[k] = ck
            allres[k] = ed[k]
            allres[k]['names'].update(cd[ck]['names'])
            version = list(allres[k]['descs'].keys())[0]
            allres[k]['descs'][version].update(cd[ck]['descs'][version])
# print(json.dumps(allres, indent = 2, ensure_ascii = False))

In [39]:
new_card_data = json.load(open('new_cards.json'))

In [40]:
for image_path, file_name, name, id, URL in new_card_data:
    id = int(id)
    keys = list(allres.keys())
    selected_keys = [x for x in keys if name in x]
    if id < 100000:
        # is charactor
        cname = ename_to_cname[selected_keys[0]].split('_')[1]
        allres['CHARACTOR/' + name] = {
          'names': {
              'en-US': name,
              'zh-CN': cname,
          },
          'image_path': image_path,
          'id': id,
        }
        continue
    if len(selected_keys) != 1:
        print('Cannot find', name, 'in', selected_keys)
        continue
    key = selected_keys[0]
    allres[key]['image_path'] = image_path
    allres[key]['id'] = id

In [41]:
json.dump(allres, open('new_result.json', 'w', encoding='utf-8'), 
          ensure_ascii=False, indent=2)